In [4]:
import math
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import os
import spacy
from spacy import displacy
from spacy.tokens import Token
from helper import *

In [5]:
#extracting dataframe paths
parent_folder = os.path.dirname(os.path.dirname(os.path.abspath("alt2.ipynb")))
char_data_path= os.path.join(parent_folder, "Data\character.metadata.tsv")
plot_data_path= os.path.join(parent_folder, "Data\plot_summaries.txt")

#building character matadata dataframe
ind={0:"Wikipedia movie ID", 1:"Freebase movie ID", 2:"Movie release date", 3:"Character name", 4:"Actor date of birth", 5:"Actor gender", 6:"Actor height", 7:"Actor ethnicity", 8:"Actor name", 9:"Actor age at movie release", 10:"Freebase character/actor map ID", 11:"Freebase character ID", 12:"Freebase actor ID"}
characters_df= pd.read_csv(char_data_path, delimiter="\t", header=None)
characters_df=characters_df.rename(columns=ind)

#building plot summaries dataframe
plots_df = pd.read_csv(plot_data_path, delimiter='\t', header=None)
plots_df = plots_df.rename(columns={0:"Wikipedia movie ID", 1:"Plot Summary"})
plots_df=plots_df.set_index("Wikipedia movie ID")

In [26]:
nlp = spacy.load("en_core_web_md")
i=1
plot=plots_df.loc[plots_df.index[i]]["Plot Summary"]

doc= nlp(plot)

displacy.render(list(doc.sents)[10])

In [7]:
#iterate over sentences
def get_all_children(token):
    children = [token]
    for child in token.children:
        children.extend(get_all_children(child))
    children=sorted(children, key=lambda x: x.i)
    return children

for sent in doc.sents:
    #print the verb and all its children and their dependency relations
    for token in sent:
        if token.pos_=="VERB":
            for child in token.children:
                if child.dep_!="punct":
                    print(token.text, child.dep_, ":", get_all_children(child))

consists nsubj : [The, nation, of, Panem]
consists prep : [of, a, wealthy, Capitol, and, twelve, poorer, districts]
provide prep : [As, punishment, for, a, past, rebellion]
provide nsubj : [each, district]
provide aux : [must]
provide dobj : [a, boy, and, girl,  ]
provide prep : [between, the, ages, of, 12, and, 18, selected, by, lottery,  , for, the, annual, Hunger, Games]
selected agent : [by, lottery,  ]
selected prep : [for, the, annual, Hunger, Games]
fight nsubj : [The, tributes]
fight aux : [must]
fight prep : [to, the, death]
fight prep : [in, an, arena]
rewarded ccomp : [The, tributes, must, fight, to, the, death, in, an, arena]
rewarded nsubjpass : [the, sole, survivor]
rewarded auxpass : [is]
rewarded prep : [with, fame, and, wealth]
chosen prep : [In, her, first, Reaping, ,, 12-year, -, old, Primrose, Everdeen]
chosen auxpass : [is]
chosen prep : [from, District, 12]
volunteers compound : [Katniss]
take nsubj : [Her, older, sister, Katniss, volunteers]
take aux : [to]
take 

In [8]:
#iterate over sentences
def get_all_children(token):
    children = [token]
    for child in token.children:
        children.extend(get_all_children(child))
    children=sorted(children, key=lambda x: x.i)
    return children

set_verb=set()
for sent in doc.sents:
    #print the verb and all its children and their dependency relations
    for token in sent:
        if token.pos_=="VERB":
            deps=[child.dep_ for child in token.children]
            set_verb=set_verb.union(set(deps))

In [9]:
sent_df=pd.DataFrame(columns=["Sentence", "Verb"] + list(set_verb))
sent_num = 0
for sent in doc.sents:
    # print the verb and all its children and their dependency relations
    sent_num += 1
    for token in sent:
        if token.pos_ == "VERB":
            # Create a dictionary with the values to be assigned
            values = {"Sentence": sent_num, "Verb": token}
            for child in token.children:
                values[child.dep_] = get_all_children(child)
            # Append the dictionary as a new row to sent_df
            sent_df.loc[len(sent_df)] = values
sent_df

,Sentence,Verb,advmod,advcl,acomp,dative,neg,agent,dobj,prt,...,auxpass,mark,prep,nsubjpass,conj,ccomp,aux,compound,cc,xcomp
0,1,consists,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,"[of, a, wealthy, Capitol, and, twelve, poorer,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,must,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,provide,NaN,NaN,NaN,NaN,NaN,NaN,"[a, boy, and, girl, ]",NaN,...,NaN,NaN,"[between, the, ages, of, 12, and, 18, selected...",NaN,NaN,NaN,[must],NaN,NaN,NaN
3,2,selected,NaN,NaN,NaN,NaN,NaN,"[by, lottery, ]",NaN,NaN,...,NaN,NaN,"[for, the, annual, Hunger, Games]",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3,must,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,51,warns,NaN,NaN,NaN,NaN,NaN,NaN,[Katniss],NaN,...,NaN,NaN,NaN,NaN,NaN,"[that, she, has, made, powerful, enemies, afte...",NaN,NaN,NaN,NaN
135,51,made,NaN,NaN,NaN,NaN,NaN,NaN,"[powerful, enemies]",NaN,...,NaN,[that],"[after, her, display, of, defiance]",NaN,NaN,NaN,[has],NaN,NaN,NaN
136,52,return,NaN,"[while, Crane, is, locked, in, a, room, with, ...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,"[to, District, 12]",NaN,"[President, Snow, considers, the, situation, .]",NaN,NaN,NaN,[and],NaN
137,52,locked,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,[is],[while],"[in, a, room, with, a, bowl, of, nightlock, be...",[Crane],NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
def process_df(df):
    # drop the punct column if present
    if "punct" in df.columns:
        df = df.drop(columns="punct")
    
    # drop aux column if present
    if "aux" in df.columns:
        df = df.drop(columns="aux")
    
    # drop auxpass column if present
    if "auxpass" in df.columns:
        df = df.drop(columns="auxpass")
    
    # drop mark column if present
    if "mark" in df.columns:
        df = df.drop(columns="mark")
    
    # drop advmod column if present
    if "advmod" in df.columns:
        df = df.drop(columns="advmod")
    
    # drop cc column if present
    if "cc" in df.columns:
        df = df.drop(columns="cc")
    
    # drop npadvmod column if present
    if "npadvmod" in df.columns:
        df = df.drop(columns="npadvmod")
    
    # drop oprd column if present
    if "oprd" in df.columns:
        df = df.drop(columns="oprd")
    
    return df

temp_df=process_df(sent_df)
temp_df

,Sentence,Verb,advcl,acomp,dative,neg,agent,dobj,prt,nsubj,prep,nsubjpass,conj,ccomp,compound,xcomp
0,1,consists,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[The, nation, of, Panem]","[of, a, wealthy, Capitol, and, twelve, poorer,...",NaN,NaN,NaN,NaN,NaN
1,2,must,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,provide,NaN,NaN,NaN,NaN,NaN,"[a, boy, and, girl, ]",NaN,"[each, district]","[between, the, ages, of, 12, and, 18, selected...",NaN,NaN,NaN,NaN,NaN
3,2,selected,NaN,NaN,NaN,NaN,"[by, lottery, ]",NaN,NaN,NaN,"[for, the, annual, Hunger, Games]",NaN,NaN,NaN,NaN,NaN
4,3,must,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,51,warns,NaN,NaN,NaN,NaN,NaN,[Katniss],NaN,[Haymitch],NaN,NaN,NaN,"[that, she, has, made, powerful, enemies, afte...",NaN,NaN
135,51,made,NaN,NaN,NaN,NaN,NaN,"[powerful, enemies]",NaN,[she],"[after, her, display, of, defiance]",NaN,NaN,NaN,NaN,NaN
136,52,return,"[while, Crane, is, locked, in, a, room, with, ...",NaN,NaN,NaN,NaN,NaN,NaN,"[She, and, Peeta]","[to, District, 12]",NaN,"[President, Snow, considers, the, situation, .]",NaN,NaN,NaN
137,52,locked,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[in, a, room, with, a, bowl, of, nightlock, be...",[Crane],NaN,NaN,NaN,NaN


In [27]:
#create an empty dataframe, with columns : Sentence, Subject, Verb, Object, Place, Time
temp_df[temp_df["pobj"].notna()].dropna(axis=1, how="all")

KeyError: 'pobj'

In [28]:

temp_df["Object"]=temp_df["dobj"]
ind_obj = temp_df[temp_df["Object"].isna()].index
temp_df.loc[ind_obj, "Object"] = temp_df.loc[ind_obj, "nsubjpass"]
ind_obj = temp_df[temp_df["Object"].isna()].index
temp_df.loc[ind_obj, "Object"] = temp_df.loc[ind_obj, "ccomp"]

temp_df["Subject"]=temp_df["nsubj"]
ind_subj = temp_df[temp_df["Subject"].isna()].index
temp_df.loc[ind_subj, "Subject"] = temp_df.loc[ind_subj, "agent"]

temp_df[["Sentence", "Verb", "Subject", "Object"]].dropna(axis=0, how="any")

,Sentence,Verb,Subject,Object
2,2,provide,"[each, district]","[a, boy, and, girl, ]"
9,5,take,"[Her, older, sister, Katniss, volunteers]","[her, place]"
10,6,gave,[who],[bread]
14,8,warns,[He],[them]
17,9,reveals,[Peeta],"[his, love, for, Katniss]"
21,10,provide,"["", sponsors, ""]","[in, -, Games, gifts, of, food, ,, medicine, ,..."
22,11,discovers,[she],"[Peeta, meant, what, he, said]"
23,11,meant,[Peeta],"[what, he, said]"
24,11,said,[he],[what]
28,12,survives,[Katniss],"[The, televised, Games, begin, with, half, of,..."


In [ ]:
def get_characters(doc):
    characters = []
    for ent in doc.ents:
        if ent.label_ == "PERSON" and ent.text.istitle():
            characters.append(ent.text)
    return list(set(characters))

characters = get_characters(doc)


def contains_character(elem, characters):
    if isinstance(elem, list):
        return any(any(char in token.text for char in characters) for token in elem)
    else:
        return False

# Apply the function to each row
fsent_df[sent_df.apply(lambda row: any(contains_character(elem, characters) for elem in row[2:]), axis=1)]

,Sentence,Verb,ccomp,advmod,punct,prep,npadvmod,acomp,xcomp,dative,...,nsubj,agent,nsubjpass,aux,neg,cc,compound,oprd,advcl,dobj
8,5,volunteers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,[Katniss],NaN,NaN,NaN
10,6,gave,NaN,[once],NaN,NaN,NaN,NaN,NaN,NaN,...,[who],NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[when, she, was, starving]","[Katniss, bread]"
12,7,taken,NaN,NaN,[.],"[to, the, Capitol]",NaN,NaN,NaN,NaN,...,NaN,NaN,"[Katniss, and, Peeta]",NaN,NaN,NaN,NaN,NaN,"[accompanied, by, their, frequently, drunk, me...",NaN
13,7,accompanied,NaN,NaN,NaN,"[past, victor, Haymitch, Abernathy]",NaN,NaN,NaN,NaN,...,NaN,"[by, their, frequently, drunk, mentor, ,]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,9,reveals,NaN,[unexpectedly],[.],"[During, a, TV, interview, with, Caesar, Flick...",NaN,NaN,NaN,NaN,...,[Peeta],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[his, love, for, Katniss]"
22,11,discovers,"[Peeta, meant, what, he, said]",[However],[.],NaN,NaN,NaN,NaN,NaN,...,[she],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,11,meant,"[what, he, said]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,[Peeta],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,12,survives,"[The, televised, Games, begin, with, half, of,...",[barely],[.],NaN,NaN,NaN,"[ignoring, Haymitch, 's, advice, to, run, away...",NaN,...,[Katniss],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,12,ignoring,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[Haymitch, 's, advice, to, run, away, from, th..."
33,13,forms,NaN,NaN,[.],NaN,NaN,NaN,NaN,NaN,...,[Peeta],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[an, uneasy, alliance, with, the, four, Careers]"


In [ ]:
for ent in doc.ents:
    print(ent.text, ent.label_)

Panem ORG
Capitol ORG
twelve CARDINAL
between the ages of 12 and 18 DATE
the annual Hunger Games EVENT
Reaping PERSON
12-year-old DATE
Primrose Everdeen FAC
12 CARDINAL
Katniss PERSON
Peeta Mellark PERSON
baker PERSON
Katniss PERSON
District 12 LOC
Katniss PERSON
Peeta PERSON
Capitol FAC
Haymitch Abernathy PERSON
Caesar Flickerman PERSON
Peeta PERSON
Katniss PERSON
Peeta PERSON
half CARDINAL
the first few minutes TIME
Katniss PERSON
Haymitch PERSON
four CARDINAL
Katniss PERSON
Katniss PERSON
Glimmer PERSON
Katniss PERSON
Katniss PERSON
a couple of days DATE
Katniss PERSON
Rue FAC
Cato PERSON
Katniss PERSON
Rue FAC
Rue FAC
District 1 LOC
Katniss PERSON
Rue FAC
Katniss PERSON
Rue FAC
Rue PERSON
Rue's FAC
11 CARDINAL
Seneca Crane PERSON
Gamemaker GPE
Katniss PERSON
Peeta PERSON
Haymitch PERSON
Crane PERSON
Katniss PERSON
Peeta PERSON
Peeta PERSON
Katniss PERSON
Thresh PERSON
District 11 ORG
Katniss PERSON
Rue FAC
Katniss PERSON
Rue FAC
Foxface ORG
District 5 LOC
Peeta PERSON
late at night